## Init For Agents Testing

In [1]:
# Setup: Load environment variables and dependencies
import os
import sys
from pathlib import Path

from jinja2 import Environment, FileSystemLoader

project_root = Path.cwd()
src_path = project_root / "src"

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
    print(f"✓ Added to sys.path: {src_path}")

from LabAgentSkill import skills_utils
from LabAgentSkill.SkillAwareAgent import SkillAwareAgent

root_dir = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
env_path = root_dir / ".env"
env = {}

if env_path.exists():
    for line in env_path.read_text().splitlines():
        line = line.strip()
        if not line or line.startswith("#") or "=" not in line:
            continue
        key, value = line.split("=", 1)
        env[key.strip()] = value.strip()

# Set API key
os.environ["OPENAI_API_KEY"] = env.get("OPENAI_API_KEY", os.environ.get("OPENAI_API_KEY", ""))
env = Environment(loader=FileSystemLoader('prompts/'))  
skills_folder = Path("/home/snt/projects_lujun/LabAgentSkill/skillsHub/skills")
all_skills = skills_utils.read_all_skills_metadata(skills_folder)
for skill in all_skills:
    print(f"  - {skill['name']}: {skill['description']}")

✓ Added to sys.path: /home/snt/projects_lujun/LabAgentSkill/src


/home/snt/projects_lujun/LabAgentSkill/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


  - langgraph-docs: Use this skill for requests related to LangGraph in order to fetch relevant documentation to provide accurate, up-to-date guidance.
  - sales-analytics: Database schema and business logic for sales data analysis including customers, orders, and revenue.
  - sentiment-analytics: Analyze text for positive and negative sentiment referencing predefined keywords.


## Load Data

In [2]:
sentiment_data = {
    "text": [
        "This movie is absolutely wonderful! I loved every minute of it.",
        "Terrible film, I couldn't even finish watching it.",
        # "The acting was decent, but the plot was confusing and boring.",
        # "Amazing cinematography and great performances. Highly recommended!",
        # "Not bad, but I've seen better. It was okay I guess.",
    ],
    "label": [1, 0, 0, 1, 0]  # 1 = Positive, 0 = Negative
}
test_samples = sentiment_data

label_map = {0: "Negative", 1: "Positive"}

# for i, (text, label) in enumerate(zip(test_samples["text"], test_samples["label"]), 1):
#     label_name = label_map[label]
#     print(f"  [{i}] {text[:65]}...")
#     print(f"      → Sentiment: {label_name}")

# print("\n" + "="*80)


## Select skills 

In [ ]:

agent_skill_aware = SkillAwareAgent(
    use_chat_history=True
)

agent_exec_skill_agent = SkillAwareAgent(
    use_chat_history=True
)

skill_inst = """**Instructions:** 
- ALWAYS identify the most suitable skill first.
- Answer ONLY with the skill name based on the query.
- If multiple skills match, answer their names separated by commas.
- If no skill matches, reply: "I didn't find the right skill."

**User Query:**"""

for i, (text, label) in enumerate(zip(test_samples["text"], test_samples["label"]), 1):

    ## 1. Select Skills
    skill_context = "\n".join([
    f"- **{skill['name']}**: {skill['description']} "
    for skill in all_skills
    ])

    template = env.get_template('system_prompt_template.jinja') 
    system_prompt = template.render(SKILL_CONTEXT=skill_context)
    task = skill_inst + f"""

    Classify the following news article as **positive** or **negative** sentiment:
                                                                                                                                                                                                                                                                    
    {text}"""
    response = agent_skill_aware.chat(user_input=task, custom_system_prompt=system_prompt)
    # agent_skill_aware.display_human_and_ai_message_history()

    ## 2. Use The Skill To Do Reasoning
    skill_meta_data_selected = skills_utils.parse_skill_from_response(response=response, skills_metadata=all_skills)
    template = env.get_template('system_prompt_exec_post_skill.jinja') 
    skill_context = ""
    for skill_meta_data in skill_meta_data_selected:
        skill_context += f"- **{skill_meta_data['name']}**: {skill_meta_data['description']}\n {skill_meta_data['body']}\n\n"

    system_prompt = template.render(SKILL_CONTEXT=skill_context)
    skill_inst = """
    **User Query:**"""

    task = skill_inst + f"""

    Classify the following news article as **positive** or **negative** sentiment: Format you final answer as "Positive" or "Negative" only, without any explanation.
                                                                                                                                                                                                                                                                    
    {text}"""
    response = agent_exec_skill_agent.chat(user_input=task, custom_system_prompt=system_prompt)


    ## 3. Evaluation
    # agent_exec_skill_agent.display_human_and_ai_message_history()

    ## 3. Parse the response and evaluate



✓ SkillAwareAgent initialized
  Model: gpt-4o-mini
  Chat History: ENABLED ✓
  Trim Messages: ENABLED ✓
✓ SkillAwareAgent initialized
  Model: gpt-4o-mini
  Chat History: ENABLED ✓
  Trim Messages: ENABLED ✓


In [5]:

agent_skill_aware.display_human_and_ai_message_history()





╔══════════════════════════════════════════════════════════════════════════════╗
║ CONVERSATION HISTORY                                                         ║
║ Total Messages: 4 | Turns: 2                                                 ║
╚══════════════════════════════════════════════════════════════════════════════╝

┌─ TURN 1 ────────────────────────────────────────────────────────────────────────
│
│ 👤 USER:
│   **Instructions:** 
│   - ALWAYS identify the most suitable skill first.
│   - Answer ONLY with the skill name based on the query.
│   - If multiple skills match, answer their names separated by commas.
│   - If no skill matches, reply: "I didn't find the right skill."
│   
│   **User Query:**
│   
│   Classify the following news article as **positive** or **negative**
│   sentiment:
│   
│       This movie is absolutely wonderful! I loved every minute of it.
│
│ 🤖 AGENT:
│   sentiment-analytics
│
└────────────────────────────────────────────────────────────────────────